In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pmdarima as pm
import os
import math
from sklearn.metrics import mean_squared_error
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Load the data

path = "/home/guts/Documents/accel_data/acceleration_data.csv"
df = pd.read_csv(path)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df

In [ ]:
# Plot the data
plt.figure(figsize=(20,10))
plt.plot(df['Timestamp'], df['Acceleration'])
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Split the data
split = int(0.8 * len(df))
train = df[18:split]
test = df[split:]

In [ ]:
# Plot the train and test sets
plt.figure(figsize=(20,10))
plt.plot(train['Timestamp'], train['Acceleration'], color="black", label="Train Set")
plt.plot(test['Timestamp'], test['Acceleration'], color="red", label="Test Set")
plt.xlabel("Date")
plt.ylabel("Value")
plt.title("Train and Test Sets")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()


In [ ]:
# Fit the model
model = pm.auto_arima(train['Acceleration'], 
                      start_p=1, start_q=1,
                      test='adf',  # Use ADF test to find optimal 'd'
                      max_p=5, max_q=5,
                      m=10,  # Frequency of series
                      start_P=0, 
                      seasonal=True, 
                      d=1,  # Apply differencing
                      D=1,  # Seasonal differencing
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)


In [ ]:
# Forecast
print(model.summary())
forecasts = model.predict(n_periods=len(test))
forecasts

In [ ]:
# Evaluate the model
forecasts_df = pd.DataFrame()
forecasts_df['ds'] = test['ds']
forecasts_df['Acceleration'] = forecasts.values
forecasts_df['ds'] = pd.to_datetime(forecasts_df['ds'])
#forecasts_with_nans = insert_nans(forecasts_df)
print((forecasts_df))
plt.figure(figsize=(30, 10))
plt.plot(forecasts_df['ds'], forecasts_df['Acceleration'], color="blue", label="Predictions")

In [ ]:
# Plot the train, test sets and predictions

plt.figure(figsize=(30,10))
plt.plot(train['ds'], train['Acceleration'], color="black", label="Train Set")
plt.plot(test['ds'], test['Acceleration'], color="red", label="Test Set")
plt.plot(forecasts_df['ds'], forecasts_df['Acceleration'], color="blue", label="Predictions")

plt.xlabel("Date")
plt.ylabel("Value")
plt.title("Train, Test Sets and Predictions")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.show()

